In [1]:
import sys
import cv2 as cv
import numpy as np

from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import QFileDialog, QMainWindow

from mainForm import Ui_MainWindow
from functools import reduce
from PIL import Image,ImageFilter

import os

def PIL_to_CV2(img):
        img2 = cv.cvtColor(np.asarray(img), cv.COLOR_RGB2BGR)
        return img2
def CV2_to_PIL(img):
        img2 = Image.fromarray(cv.cvtColor(img, cv.COLOR_BGR2RGB))
        return img2
def blur(n,img):
        out_img = img.filter(ImageFilter.BoxBlur(n))
        return out_img
def phash(img):
    """
    :param img: 圖片
    :return: 返回圖片的局部hash值
    """
    img = img.resize((8, 8), Image.ANTIALIAS).convert('L')
    avg = reduce(lambda x, y: x + y, img.getdata()) / 64.
    hash_value=reduce(lambda x, y: x | (y[1] << y[0]), enumerate(map(lambda i: 0 if i < avg else 1, img.getdata())), 0)
    print(hash_value)
    return hash_value

# 計算漢明距離:
def hamming_distance(a, b):
    """
    :param a: 圖片1的hash值
    :param b: 圖片2的hash值
    :return: 返回兩個圖片hash值的漢明距離
    """
    hm_distance=bin(a ^ b).count('1')
    print(hm_distance)
    return hm_distance

Step1 = []
Step2 = []
do_img1 = 1
do_img2 = 1


class PyQtMainEntry(QMainWindow, Ui_MainWindow):
    
    def __init__(self):
        super().__init__()
        self.setupUi(self)
        
        self.camera = cv.VideoCapture(0)
        self.is_camera_opened = False  # 摄像头有没有打开标记

        # 定时器：30ms捕获一帧
        self._timer = QtCore.QTimer(self)
        self._timer.timeout.connect(self._queryFrame)
        self._timer.setInterval(30)
        
        self.cwd = os.getcwd()#獲取當前程序文件位置
        global do_img1, do_img2
        do_img1 = 0
        do_img2 = 0
        

    def btnOpenCamera_Clicked(self):
        '''
        打开和关闭摄像头
        '''
        self.is_camera_opened = ~self.is_camera_opened
        if self.is_camera_opened:
            self.btnOpenCamera.setText("关闭摄像头")
            self._timer.start()
        else:
            self.btnOpenCamera.setText("打开摄像头")
            self._timer.stop()

    def btnCapture_Clicked(self):
        '''
        捕获图片
        '''
        # 摄像头未打开，不执行任何操作
        if not self.is_camera_opened:
            return

        self.cpatured1 = self.captured1 = self.frame
        self.cpatured2 = self.captured2 = self.frame
        

        # 后面这几行代码几乎都一样，可以尝试封装成一个函数
        rows, cols, channels = self.captured2.shape
        bytesPerLine = channels * cols
        # Qt显示图片时，需要先转换成QImgage类型
        QImg = QImage(self.captured1.data, cols, rows, bytesPerLine, QImage.Format_RGB888)
        self.labelCapture1.setPixmap(QPixmap.fromImage(QImg).scaled(
            self.labelCapture1.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
        
        rows2, cols2, channels2 = self.captured2.shape
        bytesPerLine2 = channels2 * cols2
        # Qt显示图片时，需要先转换成QImgage类型
        QImg2 = QImage(self.captured2.data, cols2, rows2, bytesPerLine2, QImage.Format_RGB888)
        self.labelCapture2.setPixmap(QPixmap.fromImage(QImg2).scaled(
            self.labelCapture2.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))

    def btnReadImage1_Clicked(self):
        '''
        从本地读取图片
        '''
        # 打开文件选取对话框
        files, filetype = QFileDialog.getOpenFileNames(self, "多文件選擇", self.cwd,"All Files(*);;PNG Files(*.png);;JPG Files(*.jpg)")
        if len(files)==0:
            print("\n取消選擇")
            return
        filenames = []
        for file in files:
            print(file)
            filenames.append(file)
        #print(filenames)
        #print(filenames[0])
        #print(filenames[1])
        self.cpatured1 = self.captured1 = cv.imread(filenames[0])
        self.cpatured2 = self.captured2 = cv.imread(filenames[1])
        
        self.cpatured1 = self.captured1 = cv.cvtColor(self.captured1, cv.COLOR_BGR2RGB)
        self.cpatured2 = self.captured2 = cv.cvtColor(self.captured2, cv.COLOR_BGR2RGB)

        rows, cols, channels = self.captured1.shape
        bytesPerLine = channels * cols
        QImg = QImage(self.captured1.data, cols, rows, bytesPerLine, QImage.Format_RGB888)
        self.labelCapture1.setPixmap(QPixmap.fromImage(QImg).scaled(
            self.labelCapture1.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
        
        rows2, cols2, channels2 = self.captured2.shape
        bytesPerLine2 = channels2 * cols2
        QImg2 = QImage(self.captured2.data, cols2, rows2, bytesPerLine2, QImage.Format_RGB888)
        self.labelCapture2.setPixmap(QPixmap.fromImage(QImg2).scaled(
            self.labelCapture2.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
        
        """
        if filename:
            self.cpatured1 = self.captured1 = cv.imread(str(filename))
            # OpenCV图像以BGR通道存储，显示时需要从BGR转到RGB
            self.cpatured1 = self.captured1 = cv.cvtColor(self.captured1, cv.COLOR_BGR2RGB)

            rows, cols, channels = self.captured1.shape
            bytesPerLine = channels * cols
            QImg = QImage(self.captured1.data, cols, rows, bytesPerLine, QImage.Format_RGB888)
            self.labelCapture1.setPixmap(QPixmap.fromImage(QImg).scaled(
                self.labelCapture1.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
        """
        
    
    def btnRotate_Clicked(self):
        if not hasattr(self, "captured1"):
            print("captured1 disappear QWQ")
            return
        if not hasattr(self, "captured2"):
            print("captured2 disappear QWQ")
            return
        #img = CV2_to_PIL(self.captured)
        #img = img.rotate(45)
        #self.cpatured = PIL_to_CV2(img)
        global do_img1, do_img2
        if(do_img1):
            img1 = Image.fromarray(self.cpatured1)
            
            if(self.degree.text()==''):
                img1 = img1.rotate(45)
            else:
                img1 = img1.rotate(int(self.degree.text()))
            
            self.cpatured1 = np.array(img1)

            rows, cols, channels = self.cpatured1.shape
            bytesPerLine = channels * cols
            QImg = QImage(self.cpatured1.data, cols, rows, bytesPerLine, QImage.Format_RGB888)
            self.labelResult1.setPixmap(QPixmap.fromImage(QImg).scaled(
            self.labelResult1.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
            Step1.append('R')
        if(do_img2):
            img2 = Image.fromarray(self.cpatured2)
            if(self.degree.text()==''):
                img2 = img2.rotate(45)
            else:
                img2 = img2.rotate(int(self.degree.text()))
            self.cpatured2 = np.array(img2)

            rows2, cols2, channels2 = self.cpatured2.shape
            bytesPerLine2 = channels2 * cols2
            QImg2 = QImage(self.cpatured2.data, cols2, rows2, bytesPerLine2, QImage.Format_RGB888)
            self.labelResult2.setPixmap(QPixmap.fromImage(QImg2).scaled(
            self.labelResult2.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
            Step2.append('R')

        
    def btnLRFlip_Clicked(self):
        if not hasattr(self, "captured1"):
            print("captured1 disappear QWQ")
            return
        if not hasattr(self, "captured2"):
            print("captured2 disappear QWQ")
            return
        
        global do_img1, do_img2
        if(do_img1):
            img1 = Image.fromarray(self.cpatured1)
            img1 = img1.transpose(Image.FLIP_LEFT_RIGHT)
            self.cpatured1 = np.array(img1)

            rows, cols, channels = self.cpatured1.shape
            bytesPerLine = channels * cols
            QImg = QImage(self.cpatured1.data, cols, rows, bytesPerLine, QImage.Format_RGB888)
            self.labelResult1.setPixmap(QPixmap.fromImage(QImg).scaled(
            self.labelResult1.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
            Step1.append('L')
        if(do_img2):
            img2 = Image.fromarray(self.cpatured2)
            img2 = img2.transpose(Image.FLIP_LEFT_RIGHT)
            self.cpatured2 = np.array(img2)

            rows2, cols2, channels2 = self.cpatured2.shape
            bytesPerLine2 = channels2 * cols2
            QImg2 = QImage(self.cpatured2.data, cols2, rows2, bytesPerLine2, QImage.Format_RGB888)
            self.labelResult2.setPixmap(QPixmap.fromImage(QImg2).scaled(
            self.labelResult2.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
            Step2.append('L')

        
        
    def btnUDFlip_Clicked(self):
        if not hasattr(self, "captured1"):
            print("captured1 disappear QWQ")
            return
        if not hasattr(self, "captured2"):
            print("captured2 disappear QWQ")
            return
        
        global do_img1, do_img2
        if(do_img1):
            img1 = Image.fromarray(self.cpatured1)
            img1 = img1.transpose(Image.FLIP_TOP_BOTTOM)
            self.cpatured1 = np.array(img1)

            rows, cols, channels = self.cpatured1.shape
            bytesPerLine = channels * cols
            QImg = QImage(self.cpatured1.data, cols, rows, bytesPerLine, QImage.Format_RGB888)
            self.labelResult1.setPixmap(QPixmap.fromImage(QImg).scaled(
            self.labelResult1.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
            Step1.append('U')
        if(do_img2):
            img2 = Image.fromarray(self.cpatured2)
            img2 = img2.transpose(Image.FLIP_TOP_BOTTOM)
            self.cpatured2 = np.array(img2)

            rows2, cols2, channels2 = self.cpatured2.shape
            bytesPerLine2 = channels2 * cols2
            QImg2 = QImage(self.cpatured2.data, cols2, rows2, bytesPerLine2, QImage.Format_RGB888)
            self.labelResult2.setPixmap(QPixmap.fromImage(QImg2).scaled(
            self.labelResult2.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
            Step2.append('U')
        
        


        
        
    def btnBlur_Clicked(self):
        # 如果没有捕获图片，则不执行操作
        if not hasattr(self, "captured1"):
            print("captured1 disappear QWQ")
            return
        if not hasattr(self, "captured2"):
            print("captured2 disappear QWQ")
            return
        #img = CV2_to_PIL(self.cpatured)
        #img_blur = blur(50,img)
        #self.cpatured = PIL_to_CV2(img_blur)
        global do_img1, do_img2
        if(do_img1):
            img1 = Image.fromarray(self.cpatured1)
            img1 = blur(20,img1)
            self.cpatured1 = np.array(img1)

            rows, cols, channels = self.cpatured1.shape
            bytesPerLine = channels * cols
            QImg = QImage(self.cpatured1.data, cols, rows, bytesPerLine, QImage.Format_RGB888)
            self.labelResult1.setPixmap(QPixmap.fromImage(QImg).scaled(
            self.labelResult1.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
            Step1.append('B')
        if(do_img2):
            img2 = Image.fromarray(self.cpatured2)
            img2 = blur(50,img2)
            self.cpatured2 = np.array(img2)

            rows2, cols2, channels2 = self.cpatured2.shape
            bytesPerLine2 = channels2 * cols2
            QImg2 = QImage(self.cpatured2.data, cols2, rows2, bytesPerLine2, QImage.Format_RGB888)
            self.labelResult2.setPixmap(QPixmap.fromImage(QImg2).scaled(
            self.labelResult2.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
            Step2.append('B')
        
        

    def btnGray_Clicked(self):
        '''
        灰度化
        '''
        # 如果没有捕获图片，则不执行操作
        if not hasattr(self, "captured1"):
            print("captured1 disappear QWQ")
            return
        if not hasattr(self, "captured2"):
            print("captured2 disappear QWQ")
            return
        global do_img1, do_img2
        if(do_img1):
            self.cpatured1 = cv.cvtColor(self.cpatured1, cv.COLOR_RGB2GRAY)

            self.cpatured1 = cv.cvtColor(self.cpatured1, cv.COLOR_GRAY2RGB)
            rows, cols, channels = self.cpatured1.shape
            bytesPerLine = channels * cols
            QImg = QImage(self.cpatured1.data, cols, rows, bytesPerLine, QImage.Format_RGB888)
            self.labelResult1.setPixmap(QPixmap.fromImage(QImg).scaled(
            self.labelResult1.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
            Step1.append('G')

        if(do_img2):
            self.cpatured2 = cv.cvtColor(self.cpatured2, cv.COLOR_RGB2GRAY)

            self.cpatured2 = cv.cvtColor(self.cpatured2, cv.COLOR_GRAY2RGB)
            rows2, cols2, channels2 = self.cpatured2.shape
            bytesPerLine2 = channels2 * cols2
            QImg2 = QImage(self.cpatured2.data, cols2, rows2, bytesPerLine2, QImage.Format_RGB888)
            self.labelResult2.setPixmap(QPixmap.fromImage(QImg2).scaled(
            self.labelResult2.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
            Step2.append('G')
        
    
    def btnNormalize_Clicked(self):
        global do_img1, do_img2
        if(do_img1):
            img = PIL_to_CV2(self.cpatured1)
            norm_img = np.zeros((800,800))
            final_img = cv.normalize(img,  norm_img, 0, 255, cv.NORM_MINMAX)
            img = CV2_to_PIL(final_img)
            self.cpatured1 = np.array(img)
            rows, cols, channels = self.cpatured1.shape
            bytesPerLine = channels * cols
            QImg = QImage(self.cpatured1.data, cols, rows, bytesPerLine, QImage.Format_RGB888)
            self.labelResult1.setPixmap(QPixmap.fromImage(QImg).scaled(
            self.labelResult1.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
            Step1.append('N')
        if(do_img2):
            img2 = PIL_to_CV2(self.cpatured2)
            norm_img2 = np.zeros((800,800))
            final_img2 = cv.normalize(img2,  norm_img2, 0, 255, cv.NORM_MINMAX)
            img2 = CV2_to_PIL(final_img2)
            self.cpatured2 = np.array(img2)
            rows2, cols2, channels2 = self.cpatured2.shape
            bytesPerLine2 = channels2 * cols2
            QImg2 = QImage(self.cpatured2.data, cols2, rows2, bytesPerLine2, QImage.Format_RGB888)
            self.labelResult2.setPixmap(QPixmap.fromImage(QImg2).scaled(
            self.labelResult2.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
            Step2.append('N')
        
    
    
        

    def btnThreshold_Clicked(self):
        '''
        Otsu自动阈值分割
        '''
        if not hasattr(self, "captured1"):
            print("captured1 disappear QWQ")
            return
        if not hasattr(self, "captured2"):
            print("captured2 disappear QWQ")
            return
        global do_img1, do_img2
        if(do_img1):
            self.cpatured1 = cv.cvtColor(self.cpatured1, cv.COLOR_RGB2GRAY)

            _, self.cpatured1 = cv.threshold(
                self.cpatured1, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

            self.cpatured1 = cv.cvtColor(self.cpatured1, cv.COLOR_GRAY2RGB)
            rows, cols, channels = self.cpatured1.shape
            bytesPerLine = channels * cols
            QImg = QImage(self.cpatured1.data, cols, rows, bytesPerLine, QImage.Format_RGB888)
            self.labelResult1.setPixmap(QPixmap.fromImage(QImg).scaled(
            self.labelResult1.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
            Step1.append('T')
        
        if(do_img2):
            self.cpatured2 = cv.cvtColor(self.cpatured2, cv.COLOR_RGB2GRAY)

            _, self.cpatured2 = cv.threshold(
                self.cpatured2, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

            self.cpatured2 = cv.cvtColor(self.cpatured2, cv.COLOR_GRAY2RGB)
            rows2, cols2, channels2 = self.cpatured2.shape
            bytesPerLine2 = channels2 * cols2
            QImg2 = QImage(self.cpatured2.data, cols2, rows2, bytesPerLine2, QImage.Format_RGB888)
            self.labelResult2.setPixmap(QPixmap.fromImage(QImg2).scaled(
            self.labelResult2.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
            Step2.append('T')
        
    def btnRedo_Clicked(self):
        global do_img1, do_img2
        self.cpatured1 = self.captured1
        
        rows, cols, channels = self.cpatured1.shape
        bytesPerLine = channels * cols
        QImg = QImage(self.cpatured1.data, cols, rows, bytesPerLine, QImage.Format_RGB888)
        self.labelResult1.setPixmap(QPixmap.fromImage(QImg).scaled(
        self.labelResult1.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
        
        
        self.cpatured2 = self.captured2
        
        rows2, cols2, channels2 = self.cpatured2.shape
        bytesPerLine2 = channels2 * cols2
        QImg2 = QImage(self.cpatured2.data, cols2, rows2, bytesPerLine2, QImage.Format_RGB888)
        self.labelResult2.setPixmap(QPixmap.fromImage(QImg2).scaled(
        self.labelResult2.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
        
        self.labelProcess1.setText('')
        self.labelProcess2.setText('')
        Step1.clear()
        Step2.clear()
    
    def btnPrintState_Clicked(self):
        length = len(Step1)
        string = ''
        length2 = len(Step2)
        string2 = ''
        for i in range(length):
            if(Step1[i]=='R'):string = string + "Rotate "
            elif(Step1[i]=='B'):string = string + "Blur "
            elif(Step1[i]=='G'):string = string + "Gray "
            elif(Step1[i]=='T'):string = string + "BinaryValue "
            elif(Step1[i]=='L'):string = string + "FlipLeftRight "
            elif(Step1[i]=='U'):string = string + "FlipTopBottom "
            elif(Step1[i]=='N'):string = string + "Normalize"
        for i in range(length2):
            if(Step2[i]=='R'):string2 = string2 + "Rotate "
            elif(Step2[i]=='B'):string2 = string2 + "Blur "
            elif(Step2[i]=='G'):string2 = string2 + "Gray "
            elif(Step2[i]=='T'):string2 = string2 + "BinaryValue "
            elif(Step2[i]=='L'):string2 = string2 + "FlipLeftRight "
            elif(Step2[i]=='U'):string2 = string2 + "FlipTopBottom "
            elif(Step2[i]=='N'):string2 = string2 + "Normalize"
                
        self.labelProcess1.setText(string)
        self.labelProcess2.setText(string2)
    
    def btnSimilar_Clicked(self):
        img1 = Image.fromarray(self.cpatured1)
        img2 = Image.fromarray(self.cpatured2)
        ham = hamming_distance(phash(img1),phash(img2))
        similarity = (1-ham/64)*100
        ss = '相似度 = '+ str(similarity) +'%'
        self.labelSimilarity.setText(ss)
    
    def onCheckBox1Clicked(self):
        global do_img1, do_img2
        if self.checkBox1.isChecked():
            do_img1 = 1
            self.labelCapture1.setStyleSheet("border: 3px solid blue;")
        else:
            do_img1 = 0
            self.labelCapture1.setStyleSheet("border: 0px solid blue;")
    def onCheckBox2Clicked(self):
        global do_img1, do_img2
        if self.checkBox2.isChecked():
            do_img2 = 1
            self.labelCapture2.setStyleSheet("border: 3px solid blue;")
        else:
            do_img2 = 0
            self.labelCapture2.setStyleSheet("border: 0px solid blue;")
    
        
    
        
        

    @QtCore.pyqtSlot()
    def _queryFrame(self):
        '''
        循环捕获图片
        '''
        ret, self.frame = self.camera.read()

        img_rows, img_cols, channels = self.frame.shape
        bytesPerLine = channels * img_cols

        cv.cvtColor(self.frame, cv.COLOR_BGR2RGB, self.frame)
        QImg = QImage(self.frame.data, img_cols, img_rows, bytesPerLine, QImage.Format_RGB888)
        self.labelCamera.setPixmap(QPixmap.fromImage(QImg).scaled(
            self.labelCamera.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))


if __name__ == "__main__":
    app = QtWidgets.QApplication(sys.argv)
    window = PyQtMainEntry()
    window.show()
    sys.exit(app.exec_())

ModuleNotFoundError: No module named 'mainForm'